<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-shirleyhan1996-hash/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**:Jinjing

**Student Number**:Han

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [5]:
import pandas as pd
url='/content/W33836-XLS-ENG.csv'
df=pd.read_csv(url)

df = df.dropna()

constant_columns = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=constant_columns)

df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed')
current_date = pd.to_datetime('today')
df['Dt_Customer'] = (current_date - df['Dt_Customer']).dt.days

df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4975,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,4129,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,4476,26,426,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,4069,26,11,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,PhD,Married,58293.0,1,0,4325,94,173,...,3,6,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,4545,46,709,...,3,4,5,0,0,0,0,0,0,0
2236,4001,1946,PhD,Together,64014.0,2,1,4065,56,406,...,2,5,7,0,0,0,1,0,0,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,4319,91,908,...,3,13,6,0,1,0,0,0,0,0
2238,8235,1956,Master,Together,69245.0,0,1,4320,8,428,...,5,10,3,0,0,0,0,0,0,0


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('MntWines', axis=1)
y = df['MntWines']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)



(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

cat_cols = ['Education', 'Marital_Status']

ct = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, drop="first"), cat_cols)
    ],
    remainder='passthrough'
)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=20
)

pipe = Pipeline(
    steps=[
        ("preprocess", ct),
        ("model", rf)
    ]
)

pipe.fit(X_train, y_train)

pipe


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=10, random_state=20))])

(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [14]:
customer_data = {col: 0 for col in X_train.columns}

customer_data['Age'] = 48
customer_data['Education'] = 'Graduation'
customer_data['Marital_Status'] = 'Married'
customer_data['Income'] = 80000
customer_data['Kidhome'] = 1
customer_data['Teenhome'] = 1
customer_data['Dt_Customer'] = (pd.to_datetime('today') - pd.to_datetime('2016-10-10')).days
customer_data['Recency'] = 43
customer_data['NumDealsPurchases'] = 2
customer_data['NumWebPurchases'] = 1
customer_data['NumCatalogPurchases'] = 0
customer_data['NumStorePurchases'] = 15
customer_data['NumWebVisitsMonth'] = 5
customer_data['AcceptedCmp1'] = 0
customer_data['AcceptedCmp2'] = 0
customer_data['AcceptedCmp3'] = 0
customer_data['AcceptedCmp4'] = 0
customer_data['AcceptedCmp5'] = 0
customer_data['Complain'] = 0

new_customer = pd.DataFrame([customer_data])[X_train.columns]
prediction = pipe.predict(new_customer)[0]
print("Predict wine spending: {:.2f}".format(prediction))

Predict wine spending: 409.31


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [21]:
from sklearn.metrics import mean_squared_error, r2_score
import math

y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

r2 = r2_score(y_test, y_pred)

print(rmse,r2)


147.50483071522186 0.7801106604600194


R2 falls between 0.7-0.9 so it means it's very good for most practical regression

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [30]:
##AI
from sklearn.model_selection import cross_val_score
import numpy as np

neg_mse_scores = cross_val_score(pipe, X_train, y_train, cv=6, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-neg_mse_scores)

print(rmse_scores)
print(rmse_scores.mean)

[144.41310211 157.98878522 146.66215158 147.5043633  168.33613912
 144.42601751]
<built-in method mean of numpy.ndarray object at 0x78e8db2400f0>


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [35]:
from sklearn.model_selection import GridSearchCV,KFold

param_grid = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [3, 6],
    'model__min_samples_leaf': [2, 4, 8]
}

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv= KFold(n_splits=5, shuffle=True, random_state=20),
    scoring='neg_mean_squared_error',###AI
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
pd.DataFrame(grid_search.cv_results_)
best_model = grid_search.best_estimator_
print(best_params)
print(best_model)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


{'model__max_depth': 5, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 50}
Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=5, min_samples_leaf=2,
                                       min_samples_split=3, n_estimators=50,
                                       random_state=20))])


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import math

lin_reg = LinearRegression()
lin_pipe = Pipeline([
    ('preprocess', ct),
    ('model', lin_reg)
])

lin_pipe.fit(X_train, y_train)

y_pred_lr = lin_pipe.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = math.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print("new RMSE: {:.2f}".format(rmse_lr))
print("new R2: {:.4f}".format(r2_lr))

from sklearn.metrics import mean_squared_error, r2_score

y_pred_rf = best_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = math.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Best RMSE: {:.2f}".format(rmse_rf))
print("Best R2: {:.4f}".format(r2_rf))

new RMSE: 180.67
new R2: 0.6701
Best RMSE: 165.19
Best R2: 0.7242


(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

In [41]:
import numpy as np
import plotly.express as px

abs_residuals_lr = np.abs(y_test - y_pred_lr)
abs_residuals_rf = np.abs(y_test - y_pred_rf)

residuals_df = pd.DataFrame({
    'LR_residual': abs_residuals_lr,
    'RF_residual': abs_residuals_rf
})

fig = px.scatter(
    residuals_df,
    x='LR_residual',
    y='RF_residual',
    labels={
        'LR_residual': 'LR residuals ',
        'RF_residual': 'RF residuals '
    },
    title='LR vs RF Scatterplot'
)

min_val = 0
max_val = 800
fig.add_shape(
    type="line",
    x0=min_val,
    y0=min_val,
    x1=max_val,
    y1=max_val,
    line=dict(color="red", width=2, dash="dash")
)

fig.show()


**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

Most points were below the line, which means the random forest model has smaller prediction errors than the linear regression model, so the RF model is better.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [53]:
quantiles = df['MntMeatProducts'].quantile([1/3, 2/3])
low_thresh = quantiles.iloc[0]
high_thresh = quantiles.iloc[1]


def meat_label(amount):
    if amount < low_thresh:
        return 'low'
    elif amount > high_thresh:
        return 'high'
    else:
        return 'medium'

df['MeatLabel'] = df['MntMeatProducts'].apply(meat_label)

print(df['MeatLabel'].value_counts())


MeatLabel
medium    746
high      739
low       731
Name: count, dtype: int64


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [54]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

categorical_cols = ['Education', 'Marital_Status']
X_cls = df.drop(columns=['MntMeatProducts', 'MeatLabel'])
y_cls = df['MeatLabel']
numeric_cols = [col for col in X_cls.columns if col not in categorical_cols]
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=20, stratify=y_cls
)
preprocess_cls = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', StandardScaler(), numeric_cols)
])
knn_model = KNeighborsClassifier(n_neighbors=5)
pipe_knn = Pipeline([
    ('preprocess', preprocess_cls),
    ('model', knn_model)
])
pipe_knn.fit(X_train_cls, y_train_cls)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status']),
                                                 ('num', StandardScaler(),
                                                  ['ID', 'Year_Birth', 'Income',
                                                   'Kidhome', 'Teenhome',
                                                   'Dt_Customer', 'Recency',
                                                   'MntWines', 'MntFruits',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth',
                                                   'AcceptedCmp3',
                                                   'AcceptedCmp4',
                                                   'AcceptedCmp5',
                                                   'AcceptedCmp1',
                                                   'AcceptedCmp2', 'Complain',
                                                   'Response'])])),
                ('model', KNeighborsClassifier())])

(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [55]:
from sklearn.metrics import classification_report

y_pred_knn = pipe_knn.predict(X_test_cls)

print(classification_report(y_test_cls, y_pred_knn))
from sklearn.metrics import classification_report



              precision    recall  f1-score   support

        high       0.84      0.82      0.83       148
         low       0.76      0.89      0.82       146
      medium       0.67      0.57      0.62       150

    accuracy                           0.76       444
   macro avg       0.76      0.76      0.76       444
weighted avg       0.76      0.76      0.75       444



From AI


Overall accuracy is 0.76, meaning the model correctly classifies about 76% of customers.
Performance is strong for high and low spenders, but weaker for the medium group, which is expected because medium is between two extremes and boundaries are less clear.
In short, the KNN model works reasonably well, especially for distinguishing high vs. low meat spenders.

# Describe how you used Gen. AI. in this assignment (2 points)

Question 1-(v) / Question 2used AI for codes and explanation

**Note**: The remaining 5 points will be assigned to readability of the work.